In [ ]:
# membuat symlink ke folder sementara
print("\n\033[92m➤ Mempersiapkan folder 🗑️\033[97m")
!rm -rf /home/studio-lab-user/content/sdw/models/Stable-diffusion/tmp_models
!rm -rf /home/studio-lab-user/content/sdw/models/Lora/tmp_Lora
!rm -rf /home/studio-lab-user/content/sdw/models/tmp_VAE
!ln -vs /tmp ~/tmp
!ln -vs /tmp/models /home/studio-lab-user/content/sdw/models/Stable-diffusion/tmp_models
!ln -vs /tmp/Lora /home/studio-lab-user/content/sdw/models/Lora/tmp_Lora
!ln -vs /tmp/VAE /home/studio-lab-user/content/sdw/models/VAE/tmp_VAE

!mkdir -p /tmp/models
!mkdir -p /tmp/Lora
!mkdir -p /tmp/VAE
# Jika Anda menemukan kesalahan dalam kode, entah itu kesalahan ketik atau hal lainnya, laporkan.
from IPython.display import clear_output
print("Mempersiapkan Vae Tambahan")
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors -d /home/studio-lab-user/content/sdw/models/VAE -o vae-ft-mse-840000-ema-pruned.safetensors
clear_output()

## =============== konfigurasi lingkungan startup ===============
import os
import time
import json
from datetime import timedelta
from subprocess import getoutput

def load_settings():
    SETTINGS_FILE = '/home/studio-lab-user/content/settings_vorstcavry.json'
    if os.path.exists(SETTINGS_FILE):
        with open(SETTINGS_FILE, 'r') as f:
            settings = json.load(f)
        return settings

settings = load_settings()
ngrok_token = settings['ngrok_token']
commandline_arguments = settings['commandline_arguments']

output = getoutput('nvidia-smi --query-gpu=gpu_name --format=csv')
if "name" in output:
    pass
else:
    commandline_arguments += " --skip-torch-cuda-test --precision full --no-half --use-cpu SD GFPGAN BSRGAN ESRGAN SCUNet CodeFormer --all"
    commandline_arguments = commandline_arguments.replace("--xformers","")

try:
    start_colab
except:
    start_colab = int(time.time())-5

if os.path.exists("/home/studio-lab-user/content/sdw/static/colabTimer.txt"):
    os.remove("/home/studio-lab-user/content/sdw/static/colabTimer.txt")
!echo -n {start_colab} > /home/studio-lab-user/content/sdw/static/colabTimer.txt

# =============== LAUNCH ===============
if not ngrok_token:
    print("You have not entered the ngrok token!, go back to `Widgets` section and insert your token. You can get it here : https://dashboard.ngrok.com/get-started/your-authtoken")

else:
    commandline_arguments += ' --ngrok ' + ngrok_token
    commandline_arguments = commandline_arguments.replace("--share","").replace("--multiple","")

    %cd /home/studio-lab-user/content/sdw
    !COMMANDLINE_ARGS="{commandline_arguments}" python launch.py

    start_colab = float(open('/home/studio-lab-user/content/sdw/static/colabTimer.txt', 'r').read())
    time_since_start = str(timedelta(seconds=time.time()-start_colab)).split('.')[0]
    print(f"\n⌚️ \033[96;1mYou have been conducting this session for -  \033[93;2m{time_since_start}\033[96m\n\n")